# Viewing the Data

In [ ]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
plt.ioff()

import blib
blib.useTheme("dark")

import src.radarkit as radarkit
import src.radarkit.chart as chart

pp = chart.ChartRHI()

In [ ]:
files = sorted(glob.glob("data/moment/20231004/*.xz"))

assert len(files) > 0, "No files found"

file = files[0]

print(f"Selected file {file}")

In [ ]:
sweep = radarkit.sweep.Sweep(file)
pp.set_data(sweep)

In [ ]:
pp.fig

In [ ]:
import src.radarkit.map as map

import importlib

In [ ]:
map = importlib.reload(map)

In [ ]:
poly = map.get('county')

In [ ]:
lines = []
w = poly['transform']['scale']
b = poly['transform']['translate']
for arc in poly['arcs']:
    lat = w[1] * arc[0][1] + b[1]
    if lat < -89:
        continue
    line = []
    point = [0, 0]
    for p in arc:
        point[0] += p[0]
        point[1] += p[1]
        lon = w[0] * point[0] + b[0]
        lat = w[1] * point[1] + b[1]
        line.append([lon, lat])
    lines.append(line)

In [ ]:
import matplotlib

import numpy as np

In [ ]:
r_earth = 6378.0
origin = (-97.44, 35.22)
view_range = 120
view_range_deg = np.degrees(view_range / r_earth)
lat_min, lat_max = origin[1] - view_range_deg, origin[1] + view_range_deg
lon_min, lon_max = origin[0] - view_range_deg, origin[0] + view_range_deg

In [ ]:
lat_min, lat_max, lon_min, lon_max

In [ ]:
# View projection matrix

rot = map.makeRotationForCoord(*origin)

def project(coords):
    a, b = coords[:, 0], coords[:, 1]
    m = r_earth * np.cos(a)
    y = r_earth * np.sin(a)
    z = m * np.cos(b)
    x = m * np.sin(b)
    p = np.array((x, y, z)).transpose()
    return np.matmul(p, rot)

def project_v2(coords):
    m = r_earth * np.cos(coords[:, 0])
    y = r_earth * np.sin(coords[:, 0])
    z = m * np.cos(coords[:, 1])
    x = m * np.sin(coords[:, 1])
    p = np.array((x, y, z)).transpose()
    return np.matmul(p, rot)


In [ ]:
county_lines = []
for points in lines:
    points = np.array(points)
    outside = np.logical_or(np.logical_or(points[:, 0] < lon_min, points[:, 0] > lon_max),
                            np.logical_or(points[:, 1] < lat_min, points[:, 1] > lat_max))
    if np.all(outside):
        continue
    county_lines.append(points)

print(len(county_lines))

In [ ]:
line = county_lines[37]
len(line)

In [ ]:
%timeit project(line)

In [ ]:
fig = plt.figure(figsize=(9, 4))
plt.plot([len(line) for line in county_lines])
fig

In [ ]:
xlim = (-160, 160)
ylim = (-90, 90)

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
for line in county_lines:
    path = project(np.radians(line))
    ax.add_line(matplotlib.lines.Line2D(path[:, 0], path[:, 1]))
ax.axis('equal')
ax.set(xlim=xlim, ylim=ylim)


In [ ]:
fig

In [ ]:
def subset_paths():
    x_min, x_max = -160, 160
    y_min, y_max = -90, 90
    county_lines = []
    for points in lines:
        points = np.array(points)
        points = project(np.radians(points))
        outside = np.logical_or(np.logical_or(points[:, 0] < x_min, points[:, 0] > x_max),
                                np.logical_or(points[:, 1] < y_min, points[:, 1] > y_max))
        if np.all(outside):
            continue
        county_lines.append(points)
    return county_lines

In [ ]:
county_lines = subset_paths()

print(len(county_lines))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
for line in county_lines:
    ax.add_line(matplotlib.lines.Line2D(line[:, 0], line[:, 1]))
ax.axis('equal')
# ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))


In [ ]:
fig

In [ ]:
plt.close()

In [ ]:
poly['bbox']